# Data-Clean (Cuong's Session)

## Import CSV file 

In [1]:
import pandas as pd
import numpy as np
import math

test_pd = pd.read_csv('SuperCenterDataNew.csv', header=None, dtype=str)
# add dtype to prevent float
test_pd.head()
print(len(test_pd))
print (test_pd.head)

36881
<bound method NDFrame.head of          0     1     2     3      4      5      6      7      8     9   ...  \
0        39    48   130   147    475   2998   4792   5478   7146  7160  ...   
1        32    48   NaN   NaN    NaN    NaN    NaN    NaN    NaN   NaN  ...   
2        48    79   232   340    396    649    911   1016   1020  1172  ...   
3        32    48    65   130    772    816   1344   3220   4445  6536  ...   
4        39   175   421   806   1025   1629   2424   3151    NaN   NaN  ...   
...     ...   ...   ...   ...    ...    ...    ...    ...    ...   ...  ...   
36876    39   875  2665  2962  12959  14070  14406  15518  16379   NaN  ...   
36877    39    41   101   346    393    413    479    522    586   635  ...   
36878  2310  4267   NaN   NaN    NaN    NaN    NaN    NaN    NaN   NaN  ...   
36879    39    48  2528   NaN    NaN    NaN    NaN    NaN    NaN   NaN  ...   
36880    32    39   205   242   1393    NaN    NaN    NaN    NaN   NaN  ...   

        66   67

## check whether the dataframe has empty rows or not 

In [2]:
def check_blank_rows(df):
    # find rows that all cells are empty or Nan
    blank_rows = df[df.apply(lambda row: row.isnull().all() or (row.astype(str).str.strip() == '').all(), axis=1)]
    
    if blank_rows.empty:
        print("✅ No empty lines.")
    else:
        print(f"⚠️ Found {len(blank_rows)} empty rows")
        print(blank_rows)

    return blank_rows

check_blank_rows(test_pd)

✅ No empty lines.


,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75


## Change the format of dataset from pandas.dataframe to pandas.Series

In [3]:
def is_not_nan(x):
    return x is not None and not (isinstance(x, float) and math.isnan(x))

def merge_and_clean(df):
    # Combine all the value into list, remove NaN cells
    def row_to_clean_list(row):
        # row là 1 Series
        items = []
        for cell in row:
            if isinstance(cell, list):
                items.extend([v for v in cell if is_not_nan(v)])
            elif is_not_nan(cell):
                items.append(int(cell))
        return items

    # Apply for each line
    return df.apply(row_to_clean_list, axis=1)

cleaned = merge_and_clean(test_pd)

# Print result
print(cleaned)
print(type(cleaned))

0        [39, 48, 130, 147, 475, 2998, 4792, 5478, 7146...
1                                                 [32, 48]
2        [48, 79, 232, 340, 396, 649, 911, 1016, 1020, ...
3        [32, 48, 65, 130, 772, 816, 1344, 3220, 4445, ...
4              [39, 175, 421, 806, 1025, 1629, 2424, 3151]
                               ...                        
36876    [39, 875, 2665, 2962, 12959, 14070, 14406, 155...
36877    [39, 41, 101, 346, 393, 413, 479, 522, 586, 63...
36878                                         [2310, 4267]
36879                                       [39, 48, 2528]
36880                             [32, 39, 205, 242, 1393]
Length: 36881, dtype: object
<class 'pandas.core.series.Series'>


In [4]:
def check_all_integers_series(series):
    # function to test integer in the list
    def is_all_int(item_list):
        if not isinstance(item_list, list):
            return False
        return all(isinstance(x, int) for x in item_list)

    # apply test
    invalid_rows = series[~series.apply(is_all_int)]

    if invalid_rows.empty:
        print("✅ All elements are lists with integer particles")
    else:
        print(f"⚠️ Found {len(invalid_rows)} invalid particles")
        print(invalid_rows)

    return invalid_rows

check_all_integers_series(cleaned)

✅ All elements are lists with integer particles


Series([], dtype: object)

### Check duplicate particles in the list of each cart 

In [5]:
def check_duplicate_elements_in_series(series):
    # Return True if the list : lst contain duplication
    def has_duplicates(lst):
        if not isinstance(lst, list):
            return False
        return len(lst) != len(set(lst))  # If exist duplication: len(set()) < len(lst)

    # Apply the check for each row
    duplicated_rows = series[series.apply(has_duplicates)]

    if duplicated_rows.empty:
        print("✅ No duplicated row found.")
    else:
        print(f"⚠️ Found {len(duplicated_rows)} list exist duplication:")
        print(duplicated_rows)

    return duplicated_rows


check_duplicate_elements_in_series(cleaned)

✅ No duplicated row found.


Series([], dtype: object)

## Function to check distribution (existence of item in each cart)

In [6]:
distribution = cleaned.explode().value_counts()

print(distribution)
top_100 = distribution.head(100)

39       21381
48       17930
38        6495
32        6267
41        4391
         ...  
15753        1
14512        1
6240         1
4533         1
16469        1
Name: count, Length: 11799, dtype: int64


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 6))
sns.barplot(x=top_100.index, y=top_100.values, color='skyblue')

plt.title('Top 100 Most Frequent Items')
plt.xlabel('Item')
plt.ylabel('Frequency')
plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

ImportError: dlopen(/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/PIL/_imaging.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/libtiff.5.dylib
  Referenced from: <DF451369-A6F9-31F9-B9A7-AEA1B609BC68> /opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/PIL/_imaging.cpython-312-darwin.so
  Reason: tried: '/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/bin/../lib/libtiff.5.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/bin/../lib/libtiff.5.dylib' (no such file), '/usr/local/lib/libtiff.5.dylib' (no such file), '/usr/lib/libtiff.5.dylib' (no such file, not in dyld cache)

In [8]:
import pandas as pd
print(cleaned)
cleaned_df = pd.DataFrame({'transaction_id': cleaned.index, 
                          'item': cleaned.values}) # Convert the data cleaned from Series -> familiar pandas format
cleaned_df


0        [39, 48, 130, 147, 475, 2998, 4792, 5478, 7146...
1                                                 [32, 48]
2        [48, 79, 232, 340, 396, 649, 911, 1016, 1020, ...
3        [32, 48, 65, 130, 772, 816, 1344, 3220, 4445, ...
4              [39, 175, 421, 806, 1025, 1629, 2424, 3151]
                               ...                        
36876    [39, 875, 2665, 2962, 12959, 14070, 14406, 155...
36877    [39, 41, 101, 346, 393, 413, 479, 522, 586, 63...
36878                                         [2310, 4267]
36879                                       [39, 48, 2528]
36880                             [32, 39, 205, 242, 1393]
Length: 36881, dtype: object


,transaction_id,item
0,0,"[39, 48, 130, 147, 475, 2998, 4792, 5478, 7146..."
1,1,"[32, 48]"
2,2,"[48, 79, 232, 340, 396, 649, 911, 1016, 1020, ..."
3,3,"[32, 48, 65, 130, 772, 816, 1344, 3220, 4445, ..."
4,4,"[39, 175, 421, 806, 1025, 1629, 2424, 3151]"
...,...,...
36876,36876,"[39, 875, 2665, 2962, 12959, 14070, 14406, 155..."
36877,36877,"[39, 41, 101, 346, 393, 413, 479, 522, 586, 63..."
36878,36878,"[2310, 4267]"
36879,36879,"[39, 48, 2528]"


# Apriori's Algorithm Implementation
- The Clean Data was storing in `cleaned_df` variable - it's better for you to have the back-up of stored cleaned csv file of this dataframe obj

## Cuong's Session (TROY-ID NUMBER)

In [ ]:
"""
Commentarty Section (Necessary)

Provide a brief commentary describing what your code is intended to implement. Focus on the main logic or algorithm behind it. You may include a high-level overview or pseudocode to outline the steps. This helps clarify your thought process before writing the actual code.

Example:

    This function sorts a list using the Bubble Sort algorithm. It repeatedly steps through the list, compares adjacent items, and swaps them if they’re in the wrong order.
    
    Pseudocode:
    for i from 0 to n:
        for j from 0 to n-i-1:
            if arr[j] > arr[j+1]:
                swap(arr[j], arr[j+1])

"""

## An's Session (TROY-ID NUMBER)

In [ ]:
"""
Commentarty Section (Necessary)

Provide a brief commentary describing what your code is intended to implement. Focus on the main logic or algorithm behind it. You may include a high-level overview or pseudocode to outline the steps. This helps clarify your thought process before writing the actual code.

Example:

    This function sorts a list using the Bubble Sort algorithm. It repeatedly steps through the list, compares adjacent items, and swaps them if they’re in the wrong order.
    
    Pseudocode:
    for i from 0 to n:
        for j from 0 to n-i-1:
            if arr[j] > arr[j+1]:
                swap(arr[j], arr[j+1])

"""

## Dung's Session (TROY-ID NUMBER)

In [ ]:
"""
Commentarty Section (Necessary)

Provide a brief commentary describing what your code is intended to implement. Focus on the main logic or algorithm behind it. You may include a high-level overview or pseudocode to outline the steps. This helps clarify your thought process before writing the actual code.

Example:

    This function sorts a list using the Bubble Sort algorithm. It repeatedly steps through the list, compares adjacent items, and swaps them if they’re in the wrong order.
    
    Pseudocode:
    for i from 0 to n:
        for j from 0 to n-i-1:
            if arr[j] > arr[j+1]:
                swap(arr[j], arr[j+1])

"""

## Dat's Session (1677589)

In [ ]:
"""
Commentarty Section (Necessary)

# My Final Target - Implement the K-Item Set generation step in Apripory Algorithm

=> Additional Constrain: the size: within a plausible size (no clear-definition)
-> the values of support -> Using Absolute Support as input
-> the minimum-support range [3,10] for the dataset


1. Conventional K-Item Set generation: (Mentioned in the course)

    L = []
    Define the target the size you want to generate: N
    Define the minimum abs support k
    For i = 1, i < N , i ++:
        Build the mapping (M) contain item-set with size i:
        if i == 1:
            
            For all record in transaction tables:
                For items in Transaction:
                    if item in M:
                        M[item] += 1
            Build the mapping L1
            For all item in M:
                if M[item] >= k :
                    L1[item] = M[item]

            L.append(L1)
            
        else:
            Build the mapping (M) contain item-set with size i:
            C_k = candidate_gen(L[i-2]) # Generating all candidate with size i > 1 (generating i - 1 items sets)

            For all record in transaction tables:
                For candidate in C_k:
                    if candidate is subset of record:
                        M[candidate] += 1
            Build the mapping Lk
            For all candidate in M:
                if M[candidate] >= k:
                    Lk[candidate] = M[candidate]
            
            If (Len(Lk) <=1):
                break
        
            L.append(Lk)
    return L

    Given: 
    candidate_gen(L_prev):
        Ck = []

        // L_prev -> a list of (k-1)- itemset
        for each pair (A,B) in L_prev:
            if A and B share the 1st (k-2) items:
                C = A U B
                all_sub_frequent = True
                for subset in combination(C):  // check all (k-1)-subsets made from C  
                    if subset not in L_prev:
                        all_sub_frequent = False
                        break
                if all_sub_frequent:
                    Ck.append(C)





"""

In [ ]:
"""
2. Improved Version based-on (Al-Maolegi & Arkok, 2014) [https://arxiv.org/pdf/1403.3948]

    Input:
            T: List of transactions
            min_sup: Minimum support threshold

    Output:
            All frequent itemsets (Answer)

    Answer = []

    L1 = find the frequent set of items with size 1 base on Transaction Table T
    Answer.append(L1)

    Define Lk the itemset with size k
    Lk-1 = L1
    For k = 2, Lk-1 != null ; k++:
        # Possible Candidate with size k & their posisble Support values
        Ck = generate_candidates(Lk-1) 
        
        For candidate in Ck:
            min_item = find item with min support in candidate based on L1

            // get transaction that contain min_item (stored in memory)
            relevant_transactions = get_transaction(min_item, L1)
            
            for t in relevant_transactions:
                include_in_t = True
                for element in candidate:
                    if element not in t:
                        include_in_t = False
                if include_in_t = True:
                    Support(Ck)+=1
        Lk = filter_candidate_by_support(Ck, min_support)
        Answer.append(Lk)
        Lk_minus_1 = Lk

    return Answer

"""

In [ ]:
"""

3. Improved Version in building the Item-Sets based-on (Lang, 2018) [https://doi.org/10.1063/1.5033777]
    Input:
        T: list of transactions
        min_sup: minimum support threshold (as percentage)

    Output:
        All frequent itemsets

    Procedure:

        1. // Step 1: Create Boolean matrix S from transaction database
        S = boolean_matrix(T)  // rows = transactions, columns = items

        2. // Step 2: Calculate support count for each item (column sum)
        For each item j in S:
            support[j] = sum(S[:, j])
        Prune items with support < min_sup_count → generate L'1

        3. // Step 3: Calculate item weight
        For each item j:
            P[j] = 1 / (support[j] / N)

        4. // Step 4: Calculate transaction weight
        For each transaction i:
            PT[i] = average(P[j]) for all items j in T[i]

        5. // Step 5: Calculate weighted support of each item
        For each item j:
            PS[j] = sum of PT[i] where item j ∈ T[i] / sum of all PT
        Prune items with PS < min_sup → generate L1

        6. // Step 6: Generate candidate 2-itemsets C2 via self-join of L1
        For each pair (i, j) in L1:
            TID[i ∩ j] = AND(S[:, i], S[:, j])
            count[i,j] = sum(TID[i ∩ j])
            PS[i,j] = sum(PT[i]) / total PT

        Generate L'2 (by count), L2 (by PS)
        Prune: L2_final = L'2 ∩ L2

        7. // Step 7: Repeat for k-itemsets (k ≥ 3)
        While Lk is not empty:
            Ck+1 = self-join(Lk)
            For each candidate c in Ck+1:
                Perform AND over all columns in c in S → count support
                Calculate PS(c)
            Generate L'k+1 and Lk+1
            Prune: L_final = L'k+1 ∩ Lk+1

        8. Stop when no new candidates can be generated
        Return all frequent itemsets

"""

In [ ]:
"""

4. Improved Version in building Item-Sets based-on (Yuan, 2017) [https://doi.org/10.1063/1.4977361]

    -> The enhancement of this -> storing the TID by using compacted vector structure
    -> Storing the occurence of object using bit-str values 
    Input:
        T: List of transactions
        min_sup: Minimum support threshold

    Output:
        All frequent itemsets (Answer)

    Procedure:

    1. // Step 1: Generate frequent 1-itemsets with transaction IDs
    L1 = find_frequent_1_itemsets(T)

    2. // Step 2: Generate candidate 2-itemsets using self-join
    C2 = join_pairs(L1)
    L2 = { c in C2 | support(c) ≥ min_sup }

    3. Initialize:
    Answer = L1 ∪ L2
    k = 3

    4. // Step 3: Iteratively generate Lk
    While L(k-1) is not empty:
    
        // 3.1: Prune infrequent items from L(k-1)
        L(k-1) = Prune(L(k-1))

        Ck = ∅
        
        // 3.2: Generate candidate k-itemsets from L(k-1)
        For each pair (Lx, Ly) in L(k-1):
            If first (k−2) items in Lx and Ly are equal AND Lx[k−1] < Ly[k−1]:
                c = union(Lx, Ly)

                // Ensure all (k−1)-subsets of c exist in L(k-1)
                If any (k−1)-subset of c is not in L(k-1):
                    skip this candidate
                Else:
                    add c to Ck

        // 3.3: Count support using TID set intersection
        Lk = []
        For each candidate c in Ck:
            c.TIDs = intersection of TID sets from subsets of c
            c.support = size of c.TIDs

            If c.support ≥ min_sup:
                Lk.append(c)

        // 3.4: Final pruning of Lk
        For each itemset Lj in Lk:
            If count of items from L1 in Lj < k:
                remove Lj from Lk

        Answer = Answer ∪ Lk
        k = k + 1

    5. Return Answer




"""

In [ ]:
import pandas as pd 
# Storing for fast backup
cleaned_df.to_csv("backup_checkpoint.csv")
